In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  8 16:09:28 2021

In [ ]:
@author: nma
"""

In [ ]:
import xarray as xr
import xesmf
from HCtFlood.kara import flood_kara
import bottleneck
import os
import numpy as np
#%%

In [ ]:
os.chdir("/home/nma/HDD/archives/IOMOM5Op/")

In [ ]:
exp_dir = '/home/nma/mom/MOM6dev/prepro/obc/'

In [ ]:
uu = xr.open_mfdataset("*.ocean_uvel.nc")
vv = xr.open_mfdataset("*.ocean_vvel.nc")

In [ ]:
grd_p = "/home/nma/mom/MOM6dev/exps/regional2/INPUT/bob_grid.nc"

In [ ]:
grid = xr.open_dataset(grd_p)

In [ ]:
ds = xr.merge([uu,vv])

In [ ]:
ds_cut = ds.sel(xu_ocean=slice(65,100), yu_ocean=slice(0,28))

s_cut['u'].isel(time=0, st_ocean=0).plot(figsize=[10,6],cmap='jet')

%

southern boundary

In [ ]:
south = xr.Dataset()
south['lon'] = grid['x'].isel(nyp=0)
south['lat'] = grid['y'].isel(nyp=0)

western boundary

In [ ]:
west = xr.Dataset()
west['lon'] = grid['x'].isel(nxp=0)
west['lat'] = grid['y'].isel(nxp=0)

%

In [ ]:
regrid_south = xesmf.Regridder(ds_cut.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'}), south, 'bilinear', 
                               locstream_out=True, periodic=False, filename='regrid_south.nc')

In [ ]:
regrid_west = xesmf.Regridder(ds_cut.rename({'xu_ocean': 'lon', 'yu_ocean': 'lat'}), west, 'bilinear', 
                              locstream_out=True, periodic=False, filename='regrid_west.nc')

In [ ]:
u_west = regrid_west(ds_cut['u'])
v_south = regrid_south(ds_cut['v'])

%

In [ ]:
drowned_u_west = u_west.ffill(dim='nyp').ffill(dim='st_ocean')

In [ ]:
drowned_v_south = v_south.ffill(dim='nxp').ffill(dim='st_ocean')

rowned_v_south.isel(time=0).plot(figsize=[8, 6], yincrease=False, cmap='jet')

% adding dim

d= drowned_u_west.expand_dims({"nxp":drowned_u_west.lon[0]})

In [ ]:
n_west_array = np.expand_dims(drowned_u_west.values,axis=3)

In [ ]:
n_south_array = np.expand_dims(drowned_v_south.values,axis=2)

z_vals = 

%

In [ ]:
west_final = xr.Dataset({
    "u" : (["time","zl","yh","xq"],n_west_array),},
    coords = {"time":(["time",],drowned_u_west.time.values),
              "zl":(["zl",],drowned_u_west.st_ocean.values),
              "yh":(["yh",],drowned_u_west.lat.values),
              "xq":(["xq"],[drowned_u_west.lon.values[0]])})

In [ ]:
south_final = xr.Dataset({
    "v" : (["time","zl","yq","xh"],n_south_array),},
    coords = {"time":(["time",],drowned_v_south.time.values),
              "zl":(["zl",],drowned_v_south.st_ocean.values),
              "yq":(["yq",],[drowned_v_south.lat.values[0]]),
              "xh":(["xh"],drowned_v_south.lon.values)})

%

In [ ]:
west_final.to_netcdf(exp_dir+"section_west2.nc")

In [ ]:
south_final.to_netcdf(exp_dir+"section_south2.nc")